In [1]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

In [2]:
model = Model("Ubicacion fabricas")

In [3]:
# Matriz con variables exogenas
b = np.array([[2, 4, 3]
              ,[5, 3, 6]
              ,[3, 4, 2]])

max_produccion = [200, 400, 300]

max_contaminacion = [150, 250, 200]

penalizacion_contaminacion = [200, 150, 100]

contaminacion = [0.5, 2, 1]

In [4]:
# Creamos las variables
y = np.empty(3, dtype=object)
for i in range(3):
    y[i] = model.addVar(name=f"y_{i+1}", vtype="B")

x = np.empty((3,3), dtype=object)
for i in range(3):
    for j in range(3):
        x[i,j] = model.addVar(name=f"x_{i+1}_{j+1}", vtype="I", lb=0)

In [5]:
# Agregamos restricciones
model.addCons(sum(y[j] for j in range(3)) == 1, name="elegir una ubicacion")

for j in range(3):
    model.addCons(sum(x[i,j] for i in range(3)) <= max_produccion[j] * y[j], name=f"max_produccion_{j+1}")

model.addCons(sum(penalizacion_contaminacion[j]*(sum(x[i,j] - max_contaminacion[j] for i in range(3))) for j in range(3)) <= 90000
              , name="max_penalizacion")

max_penalizacion

In [6]:
# funcion objetivo
model.setObjective(sum(b[i,j]*x[i,j] for i in range(3) for j in range(3)), "maximize")

In [7]:
# Optimizar
model.redirectOutput()
model.optimize()

sol = model.getBestSol()

# Imprimimos la solucion
for v in model.getVars():
    print(f"{v.name}: {model.getSolVal(sol, v)}")

presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 3, exhaustive) 6 del vars, 0 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
(round 4, fast)       9 del vars, 4 del conss, 0 add conss, 9 chg bounds, 0 chg sides, 0 chg coeffs, 1 upgd conss, 0 impls, 1 clqs
presolving (5 rounds: 5 fast, 3 medium, 3 exhaustive):
 12 deleted vars, 5 deleted constraints, 0 added constraints, 9 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +1.80000000000000e+0

In [8]:
model.writeSol(sol, 'solucion_fabricas.sol')